In [ ]:
print("Loading modules...")

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# pip install peakutils
import peakutils

%matplotlib inline

print("Done.")

In [ ]:
print("Defining utility functions to find corners in chessboard images and to calculate the perspective transform...")

''' Utility function that takes a path to an image and attempts to find the chessboard corners on it, which will be later used to correct for camera distortion '''
def findCorners(image_path, nx, ny, objpoints, imgpoints):
    image = cv2.imread(image_path)
    image_chess = []
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If found, draw corners and append them to the list
    if ret == True:
        image_chess = np.copy(image)
        
        cv2.drawChessboardCorners(image_chess, (nx, ny), corners, ret)
        
        objp = np.zeros((nx*ny, 3), np.float32)
        objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1, 2)
        
        objpoints.append(objp)
        imgpoints.append(corners)
    
    return ret, image, image_chess, objpoints, imgpoints

''' Utility function that takes a list of source and destination points and returns the perspective matrix and its inverse '''
def calculatePerspectiveTransform(src, dst):    
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return M, M_inv

print("Done.")

In [ ]:
print("Define a utility class to hold a group of detected points...")

''' Utility class which holds a list of x and y points and some utility values and functions to perform on them '''
class LineData():
    def __init__(self, x, y):
        assert (x != None and y != None)
        assert (len(x) == len(y))
        self.x = np.array(x, dtype='float')
        self.y = np.array(y, dtype='float')
        
        self.reset()
    
    '''
    Calculates parameters of the x, y data:
    Number of pairs, x mean and standard deviation, y range, and the x most probable value.
    '''
    def reset(self):
        self.num = len(self.x)
        self.empty = (self.num == 0)
        
        self.x_mean = np.mean(self.x)
        self.x_std = np.std(self.x)
        self.y_min = np.amin(self.y) if not self.empty else None
        self.y_max = np.amax(self.y) if not self.empty else None
        
        histo, bin_edges = np.histogram(self.x, bins=10)
        max_histo = np.max(histo)
        max_index = np.argmax(histo)
        
        if (max_histo > 2):
            self.x_max = (bin_edges[max_index+1] + bin_edges[max_index]) / 2
        else:
            self.x_max = self.x_mean
    
    ''' Append x and y points from a list of other LineData objects. Recalculates all values when called. '''
    def append(self, line_data_list):
        for line_data in line_data_list:
            self.x = np.concatenate((self.x, line_data.x))
            self.y = np.concatenate((self.y, line_data.y))
        
        self.reset()

    ''' Performs a 2 degree fit on the x,y data. Takes a meter per pixel optional argument in x and y. '''
    def get_fit(self, m_per_pix = (None, None)):
        xm_per_pix = m_per_pix[0]
        ym_per_pix = m_per_pix[1]
        
        if self.num < 2:
            return None
        
        if (xm_per_pix == None):
            return np.polyfit(self.y, self.x, 2)
        else:
            return np.polyfit(self.y * ym_per_pix, self.x * xm_per_pix, 2)
    
    ''' Returns a new LineData object that has outliers removed by using the x mean, standard deviation and most probable value to do the cleanup '''
    def get_clean(self):
        x_clean = []
        y_clean = []
        # print (self.num, self.x_mean, self.x_std, self.x_max)
        epsilon = np.finfo(float).eps
        if (self.num < 3 or self.x_std < epsilon):
            return LineData(self.x, self.y)
        for i in range(self.num):
            if (abs(self.x[i] - self.x_max) < 2 * self.x_std):
            #if (abs(self.x[i] - self.x_mean) < 2 * self.x_std):
                x_clean.append(self.x[i])
                y_clean.append(self.y[i])
        
        return LineData(x_clean, y_clean)

print("Done.")

In [ ]:
print("Define a utility class to hold information about a single line...")

''' Utility class that holds information about a single line (left or right) and provides some utility functions to perform on it '''
class Line():
    def __init__(self, x_size, y_size, xm_per_pix, ym_per_pix, num_y_vals, x_begin, line_list, line_data_full_all, line_data_bottom, line_data_full):
        assert line_list != None
        assert line_data_full_all != None and line_data_full != None
        
        self.x_size = x_size
        
        self.xm_per_pix = xm_per_pix
        self.ym_per_pix = ym_per_pix
        
        self.yvals = np.linspace(0, y_size, num=num_y_vals)
        self.y_vals_m = self.yvals * self.ym_per_pix # In meters
        
        self.x_begin = x_begin
        self.line_list = line_list
                        
        self.line_data_full_all = line_data_full_all
        self.line_data_bottom = line_data_bottom
        self.line_data_full = line_data_full
        
        self.raw_fit = None # Fit on line_data_full_all
        self.smooth_fit = None # Low-pass filtered fit
        self.smooth_fit_m = None # Low-pass filtered fit in meters
        
        self.detected = (self.line_data_full_all.num > 2)
        
        self.value_at_bottom = None
        self.value_at_middle = None
        self.bottom_offset = None
        self.value_at_bottom_m = None
        self.value_at_middle_m = None
        self.bottom_offset_m = None
        self.radius_of_curvature = None
    
    ''' Utility function that returns the goodness of fit '''
    def goodness_of_fit(self, x, y, fit):
        fitx = fit[0] * y**2 + fit[1] * y + fit[2]
        mean_x = np.mean(x)

        # residual sum of squares
        ss_res = np.sum((x - fitx) ** 2)
        
        # total sum of squares
        ss_tot = np.sum((x - mean_x) ** 2)

        # r-squared
        r2 = 1 - (ss_res / ss_tot)
        
        return r2
    
    '''
    Performs a fit on the current line data and uses information from the previous frames (if it exists) to smooth the fit, using a low pass filter.
    Calculates the fit in pixels and in meters, and also the radius of curvature and the offset at the bottom of the frame.
    '''
    def fit(self, low_pass_a):
        self.raw_fit = self.line_data_full_all.get_fit()
        self.smooth_fit = self.raw_fit if len(self.line_list) == 0 else (low_pass_a * self.raw_fit + (1 - low_pass_a) * self.line_list[-1].smooth_fit)
        
        raw_fit_m = self.line_data_full_all.get_fit((self.xm_per_pix, self.ym_per_pix))
        self.smooth_fit_m = raw_fit_m if len(self.line_list) == 0 else (low_pass_a * raw_fit_m + (1 - low_pass_a) * self.line_list[-1].smooth_fit_m)
        
        self.current_fit = self.smooth_fit
        self.current_fitx = self.current_fit[0] * self.yvals**2 + self.current_fit[1] * self.yvals + self.current_fit[2]
                  
        current_fit_m = self.smooth_fit_m
        current_fitx_m = current_fit_m[0] * self.y_vals_m**2 + current_fit_m[1] * self.y_vals_m + current_fit_m[2]
        
        self.value_at_bottom = self.current_fitx[-1]
        self.value_at_middle = self.current_fitx[np.int(len(self.yvals) / 2)]
        self.bottom_offset = self.x_size / 2 - self.value_at_bottom
        
        self.value_at_bottom_m = current_fitx_m[-1]
        self.value_at_middle_m = current_fitx_m[np.int(len(self.y_vals_m) / 2)]
        self.bottom_offset_m = (self.x_size / 2) * self.xm_per_pix - self.value_at_bottom_m
        
        # Evaluate radius at bottom
        self.radius_of_curvature = ((1 + (2 * current_fit_m[0] * self.y_vals_m[-1] + current_fit_m[1])**2)**1.5) / np.absolute(2*current_fit_m[0])
    
    ''' Utility function to draw the detected peaks found during detection on a given image in the input '''
    def draw_peaks(self, image, radius, color, thickness_all, thickness_full):
        for i in range(self.line_data_full_all.num):
            x = np.int(self.line_data_full_all.x[i])
            y = np.int(self.line_data_full_all.y[i])
            #cv2.circle(image, (x, y), radius, color, thickness_all)
        for i in range(self.line_data_full.num):
            x = np.int(self.line_data_full.x[i])
            y = np.int(self.line_data_full.y[i])
            cv2.circle(image, (x, y), radius, color, thickness_full)
    
    ''' Utility function to draw the current fit on a given image in the input '''
    def draw_fit(self, image, color, thickness):
        for i in range(len(self.current_fitx) - 1):
            x1 = np.int(self.current_fitx[i])
            x2 = np.int(self.current_fitx[i + 1])
            y1 = np.int(self.yvals[i])
            y2 = np.int(self.yvals[i + 1])
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)

print("Done.")

In [ ]:
print("Define a utility class to process a single frame...")

'''
Utility class to process take a single frame of input and process it
to detect left and right lines.

Accepts an optional line_list parameter that contains a set of detected lines
in the previous frames and which can be useful in the context of a video to
help detecttion in the current frame.
'''
class Frame:
    def __init__(self, image, title="", line_list=None):
        self.image = image
        self.title = title
        self.x_size = image.shape[1]
        self.y_size = image.shape[0]
        self.line_list_l = [x[0] for x in line_list] if line_list != None else []
        self.line_list_r = [x[1] for x in line_list] if line_list != None else []
        
        self.undistorted = None
        self.warped = None
        self.sobelx_binary = None
        self.sobel_angle_binary = None
        self.h_binary = None
        self.l_binary = None
        self.s_binary = None
        self.combined_binary = None
        self.warped_fit = None
        self.histogram = None
        self.result = None
        
        self.l_line = None
        self.r_line = None
    
    ''' Utility function to create a 3 channel image from a single channel image '''
    def create_empty_3_chan_for_1_chan(self, image):
        empty_one_channel = np.zeros_like(image).astype(np.uint8)
        empty_three_channels = np.dstack((empty_one_channel, empty_one_channel, empty_one_channel))
        
        return empty_three_channels
    
    ''' Utility function to draw the zone between the detected lines as single lane '''
    def draw_fit_area(self, image, color):
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([self.l_line.current_fitx, self.l_line.yvals]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([self.r_line.current_fitx, self.l_line.yvals])))])
        pts = np.hstack((pts_left, pts_right))
        
        cv2.fillPoly(image, np.int_([pts]), color)
    
    ''' Utility function to draw the lane width information (text and arrow between the two detected lines) '''
    def draw_width_info(self, image, font_face, font_scale, color, thickness, thickness_arrow):
        lane_width_middle_m = self.r_line.value_at_middle_m - self.l_line.value_at_middle_m
        
        middle_text = "{0:5.2f}m".format(lane_width_middle_m)
        text_size, base_line = cv2.getTextSize(middle_text, font_face, font_scale, thickness) 
        
        y_ref = np.int(self.y_size / 2)
        x_middle = np.int((self.l_line.value_at_middle + self.r_line.value_at_middle) / 2 - text_size[0] / 2)
        middle_pos = (x_middle, y_ref + text_size[1] + base_line)
        cv2.putText(image, middle_text, middle_pos, font_face, font_scale, color, thickness, cv2.LINE_AA)

        # Draw two arrows in order to have two arrowheads
        l_x = np.int(self.l_line.value_at_middle)
        r_x = np.int(self.r_line.value_at_middle)
        cv2.arrowedLine(image, (l_x, y_ref), (r_x, y_ref), color, thickness_arrow)
        cv2.arrowedLine(image, (r_x, y_ref), (l_x, y_ref), color, thickness_arrow)
        
    ''' Utility function to draw the lane center information and the current offset with regard to it '''
    def draw_offset_info(self, image, font_face, font_scale, color, thickness, thickness_line, tick_height):
        lane_center_bottom = np.int((self.r_line.value_at_bottom + self.l_line.value_at_bottom) / 2)
        half_x_size = np.int(self.x_size / 2)
        car_offset_from_center_m = (self.l_line.bottom_offset_m + self.r_line.bottom_offset_m) / 2
        
        half_tick_height = np.int(tick_height / 2)
        quarter_tick_height = np.int(tick_height / 4)
        
        thickness_half_line = np.int(thickness_line / 2)
        thickness_arrow = thickness_half_line
        
        # The big tick represents the center of the lane
        pos_big_tick_start = (lane_center_bottom, self.y_size)
        pos_big_tick_end = tuple(np.subtract(pos_big_tick_start, (0, tick_height)))
        
        # The small tick represents the car (center of the image)
        pos_small_tick_start = (half_x_size, self.y_size)
        pos_small_tick_end = tuple(np.subtract(pos_small_tick_start, (0, half_tick_height)))
        
        # Draw an arrow from the big tick to the small tick to represent how much the car is offset from the center
        # Stop arrow just before the small tick starts
        cut_arrow = thickness_half_line if (car_offset_from_center_m > 0) else -thickness_half_line
        pos_arrow_start = tuple(np.subtract(pos_big_tick_start, (0, quarter_tick_height)))
        pos_arrow_end = tuple(np.subtract(pos_small_tick_start, (cut_arrow, quarter_tick_height)))
        
        offset_text = "{0:5.2f}m".format(car_offset_from_center_m)
        text_size, base_line = cv2.getTextSize(offset_text, font_face, font_scale, thickness) 
        
        pos_text = tuple(np.subtract(pos_small_tick_end, (np.int(text_size[0] / 2), half_tick_height + base_line)))
        
        cv2.line(image, pos_big_tick_start, pos_big_tick_end, color, thickness_line)
        cv2.line(image, pos_small_tick_start, pos_small_tick_end, color, thickness_half_line)
        cv2.arrowedLine(image, pos_arrow_start, pos_arrow_end, color, thickness_arrow)
        cv2.putText(image, offset_text, pos_text, font_face, font_scale, color, thickness, cv2.LINE_AA)
    
    ''' Utility function to draw the lane radius information '''
    def draw_radii_info(self, image, font_face, font_scale, color, thickness):
        radii_text = "RadiusL={0:7.2f}m RadiusR={1:7.2f}m".format(self.l_line.radius_of_curvature, self.r_line.radius_of_curvature)
        text_size, base_line = cv2.getTextSize(radii_text, font_face, font_scale, thickness) 
        
        pos_text = (self.x_size - text_size[0], text_size[1] + base_line)
        
        cv2.putText(self.result, radii_text, pos_text, font_face, font_scale, color, thickness, cv2.LINE_AA)
    
    ''' Utility function to draw the detection windows for a given line detection pass '''
    def draw_windows(self, image, windows, color, thickness):
        for window in windows:
            cv2.rectangle(image, window[0], window[1], color, thickness)
    
    ''' Utility function to draw debug information (intermediate frames used for detection) on the final result image '''
    def stack_thumbnails(self):
        num_cols = 1
        thumbnail_scale = 0.15
        thumbnail_size = (np.int(self.x_size * thumbnail_scale), np.int(self.y_size * thumbnail_scale))
        
        #l_gray = np.dstack((self.l, self.l, self.l))
        #s_gray = np.dstack((self.s, self.s, self.s))
        
        #l_binary_gray = 255 * np.dstack((self.l_binary, self.l_binary, self.l_binary))
        #s_binary_gray = 255 * np.dstack((self.s_binary, self.s_binary, self.s_binary))
        
        #sobelx_binary_gray = 255 * np.dstack((self.sobelx_binary, self.sobelx_binary, self.sobelx_binary))
        #sobel_angle_binary_gray = 255 * np.dstack((self.sobel_angle_binary, self.sobel_angle_binary, self.sobel_angle_binary))
        
        combined_binary_gray = 255 * np.dstack((self.combined_binary, self.combined_binary, self.combined_binary))        
        
        thumbnails = []
        
        thumbnails.append(cv2.resize(self.warped, thumbnail_size, interpolation=cv2.INTER_AREA))
        #thumbnails.append(None)
        
        #thumbnails.append(cv2.resize(sobelx_binary_gray, thumbnail_size, interpolation=cv2.INTER_AREA))
        #thumbnails.append(cv2.resize(sobel_angle_binary_gray, thumbnail_size, interpolation=cv2.INTER_AREA))
        #thumbnails.append(cv2.resize(l_binary_gray, thumbnail_size, interpolation=cv2.INTER_AREA))
        #thumbnails.append(cv2.resize(s_binary_gray, thumbnail_size, interpolation=cv2.INTER_AREA))
        
        thumbnails.append(cv2.resize(combined_binary_gray, thumbnail_size, interpolation=cv2.INTER_AREA))
        thumbnails.append(cv2.resize(self.warped_fit, thumbnail_size, interpolation=cv2.INTER_AREA))
        
        for i in range(len(thumbnails)):
            if (thumbnails[i] == None):
                continue
            
            x_offset = thumbnail_size[0] * (i % num_cols)
            y_offset = thumbnail_size[1] * np.int(i / num_cols)
            
            self.result[y_offset:y_offset+thumbnail_size[1], x_offset:x_offset+thumbnail_size[0]] = thumbnails[i]
    
    '''
    This function takes an image and processes it using sliding windows to detect two lines, left and right.
    
    The search zone and width of the search windows is determined by the availability of prior information on the position of the lines.
    Therefore, we can execute a blind search when no prior information is available, or use the prior information to execute a more narrow search
    around existing start positions or lien fits.
    
    The width of sliding windows is smaller when a fit is available.
    '''
    def find_peaks(self, image_to_search, search_height, stride, y_top, search_width, x_begin=(None, None), fits=(None, None)):
        assert stride <= search_height
        
        ''' This function will return the search range for the left and right lines dependinng on the previous detected value and whether we have a previous fit '''
        def calc_window_size(x_previous, y, search_width, x_interval, fit):
            assert x_previous != None
            
            begin, end = None, None
            if (fit != None):
                x_fit = np.int(fit[0] * y**2 + fit[1] * y + fit[2])
                half_search_width = np.int(search_width / 2)
                begin, end = x_fit - half_search_width, x_fit + half_search_width
            else:
                begin, end = x_previous - search_width, x_previous + search_width
            
            assert begin <= end
            
            # Adjust for out-of-range values
            if (end < x_interval[0]):
                begin = end = x_interval[0]
            elif begin < x_interval[0]:
                begin = x_interval[0]
            
            if (begin > x_interval[1]):
                begin = end = x_interval[1]
            elif (end > x_interval[1]):
                end = x_interval[1]
            
            return (begin, end)
        
        blind_search = (x_begin[0] == None or x_begin[1] == None)
                
        x_previous_l = np.int(x_begin[0]) if x_begin[0] != None else None
        x_previous_r = np.int(x_begin[1]) if x_begin[1] != None else None

        x_l, y_l, x_r, y_r = [], [], [], []
        windows_l, windows_r = [], []
                
        y_begin = self.y_size - search_height
        y_end = y_top
        assert y_begin >= y_end
        # TODO use y_current, not y_begin and maybe a for loop?
        while y_begin >= y_end:
            histogram = np.sum(image_to_search[y_begin:y_begin+search_height,:], axis=0)

            y_current = y_begin + search_height / 2

            x_interval = (0, self.x_size)
            (l_start, l_end) = (np.int(self.x_size / 4), np.int(self.x_size / 2)) if blind_search else calc_window_size(x_previous_l, y_current, search_width, x_interval, fits[0])
            (r_start, r_end) = (l_end, np.int(self.x_size * (3./4))) if blind_search else calc_window_size(x_previous_r, y_current, search_width, x_interval, fits[1])
            
            windows_l.append(((l_start, y_begin), (l_end, y_begin + search_height)))
            windows_r.append(((r_start, y_begin), (r_end, y_begin + search_height)))

            histo_left = histogram[l_start:l_end]
            histo_right = histogram[r_start:r_end]
            
            if (len(histo_left) == 0):
                print ("Left histogram is empty! [{0}, {1}] {2}".format(l_start, l_end, histo_left))
            if (len(histo_right) == 0):
                print ("Right histogram is empty! [{0}, {1}] {2}".format(r_start, r_end, histo_right))
            
            indexes_l = peakutils.indexes(histo_left, thres=0.1, min_dist=l_end-l_start) if len(histo_left) > 0 else []
            indexes_r = peakutils.indexes(histo_right, thres=0.1, min_dist=r_end-r_start) if len(histo_right) > 0 else []
            
            if(len(indexes_l) > 0):
                x_peak_l = indexes_l[-1] + l_start
                x_l.append(x_peak_l)
                y_l.append(y_current)
                x_previous_l = np.int(x_peak_l)
                
            if(len(indexes_r) > 0):
                x_peak_r = indexes_r[0] + r_start
                x_r.append(x_peak_r)
                y_r.append(y_current)
                x_previous_r = np.int(x_peak_r)
            
            y_begin = y_begin - stride
        
        return LineData(x_l, y_l), LineData(x_r, y_r), windows_l, windows_r
    
    '''
    This is the main function that processes the current frame to detect lines.
    It is controlled by a list of parameters provided as input.
    
    It calculates intermediate thresholded frames (sobel and colour channels), combines them, and performs a line detection on the combined frame.
    If it exists, it will use the lines information from the previous frames as a detection guide.
    It will then output a result frame that contains the information about the detected lines.
    '''
    def process(self, mtx, dist, M, M_inv, params):
        self.image_undistorted = cv2.undistort(self.image, mtx, dist, None, mtx)        
        self.warped = cv2.warpPerspective(self.image_undistorted, M, (self.x_size, self.y_size), flags=cv2.INTER_LINEAR)
        
        image_to_process = self.warped
        
        gray = cv2.cvtColor(image_to_process, cv2.COLOR_RGB2GRAY)
        
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=params.sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=params.sobel_kernel)
        
        # Absolute derivatives to accentuate strong lines
        sobelx_absolute = np.absolute(sobelx)
        sobely_absolute = np.absolute(sobely)
        
        # Threshold x gradient
        sobelx_absolute_scaled = (255 * sobelx_absolute / np.max(sobelx_absolute))#.astype(np.uint8)
        self.sobelx_binary = np.zeros_like(sobelx_absolute_scaled)
        self.sobelx_binary[(sobelx_absolute_scaled >= params.sobelx_threshold[0]) & (sobelx_absolute_scaled <= params.sobelx_threshold[1])] = 1
        
        # Threshold gradient angle
        sobel_angle = np.arctan2(sobely_absolute, sobelx_absolute)
        self.sobel_angle_binary = np.zeros_like(sobel_angle).astype(np.uint8)
        self.sobel_angle_binary[(sobel_angle >= params.angle_thresh[0]) & (sobel_angle <= params.angle_thresh[1])] = 1
        
        # Extract the H, L and S channels
        hls = cv2.cvtColor(image_to_process, cv2.COLOR_RGB2HLS)
        h = hls[:,:,0]
        l = hls[:,:,1]
        s = hls[:,:,2]
        
        # Histogram equalization to attenuate lighting differences
        h_eq = cv2.equalizeHist(h)
        l_eq = cv2.equalizeHist(l)
        s_eq = cv2.equalizeHist(s)

        # Threshold color channels
        self.h_binary = np.zeros_like(h_eq)
        self.h_binary[(h_eq >= params.h_threshold[0]) & (h_eq <= params.h_threshold[1])] = 1
        self.l_binary = np.zeros_like(l_eq)
        self.l_binary[(l_eq >= params.l_threshold[0]) & (l_eq <= params.l_threshold[1])] = 1
        self.s_binary = np.zeros_like(s_eq)
        self.s_binary[(s_eq >= params.s_threshold[0]) & (s_eq <= params.s_threshold[1])] = 1
        
        # Combine the binary thresholds
        self.combined_binary = np.zeros_like(self.sobelx_binary)
        self.combined_binary[((self.l_binary == 1) | (self.s_binary == 1) | (self.sobelx_binary == 1)) & (self.sobel_angle_binary == 1)] = 1
        
        image_to_fit = self.combined_binary
        
        # Draw peaks and fit lines
        self.warped_fit = 255 * np.dstack((image_to_fit, image_to_fit, image_to_fit)).astype(np.uint8)
        
        # Do a first pass to find where the lanes begin
        x_begin_l, x_begin_r = None, None
        line_data_bottom_l, line_data_bottom_r = None, None
        fit_l, fit_r = None, None
        
        if (len(self.line_list_l) < params.max_prev_frames):
            # We're still in the first few frames, we don't have enough confidence in the starting positions of the lines, so we'll look for them here.
            window_size_cur = np.int(self.y_size / 3) 
            y_top = self.y_size - window_size_cur #these vals need to go into the params
            stride_cur = np.int(params.stride_ratio * window_size_cur)
            line_data_bottom_l, line_data_bottom_r, windows_l, windows_r = self.find_peaks(image_to_fit, window_size_cur, stride_cur, y_top, params.search_width)
            # This should give us exactly one point
            assert line_data_bottom_l.num == 1 and line_data_bottom_r.num == 1
            
            # Use the current detected points, along with the ones from the previous frames (if any), to exclude any outliers and improve the detection of the starting point
            line_data_bottom_all_l = LineData(line_data_bottom_l.x, line_data_bottom_l.y)
            line_data_bottom_all_r = LineData(line_data_bottom_r.x, line_data_bottom_r.y)
            line_data_bottom_all_l.append([line.line_data_bottom for line in self.line_list_l])
            line_data_bottom_all_r.append([line.line_data_bottom for line in self.line_list_r])

            # Clean outliers
            # We expect the lines to be almost straight here.
            line_data_bottom_all_l = line_data_bottom_all_l.get_clean()
            line_data_bottom_all_r = line_data_bottom_all_r.get_clean()
            
            # These will act as the starting positions of the lines
            x_begin_l = np.int(line_data_bottom_all_l.x_max)
            x_begin_r = np.int(line_data_bottom_all_r.x_max)
            
            # For display purposes
            self.histogram = np.sum(image_to_fit[self.y_size-window_size_cur:self.y_size,:], axis=0)
        else:
            # We have enough previous frames to be confident in the starting position of the lines. We'll therefore use them.
            x_begin_l = self.line_list_l[-1].x_begin
            x_begin_r = self.line_list_r[-1].x_begin
                        
            # We'll also use the last filtered fit as a guide while detecting points on the full image
            fit_l, fit_r = self.line_list_l[-1].smooth_fit, self.line_list_r[-1].smooth_fit
        
        y_top = 0
        window_size_cur = np.int(self.y_size * params.window_size_ratio)
        stride_cur = np.int(params.stride_ratio * window_size_cur)
        line_data_full_l, line_data_full_r, windows_l, windows_r = self.find_peaks(image_to_fit, window_size_cur, stride_cur, y_top, params.search_width, x_begin=(x_begin_l, x_begin_r), fits=(fit_l, fit_r))
        line_data_full_all_l = LineData(line_data_full_l.x, line_data_full_l.y)
        line_data_full_all_r = LineData(line_data_full_r.x, line_data_full_r.y)
        line_data_full_all_l.append([line.line_data_full for line in self.line_list_l])
        line_data_full_all_r.append([line.line_data_full for line in self.line_list_r])
        
        '''
        At this point, we have a list of detected points for the 2 lines.
        We want to increase the chances of good detection by using the strongest line to 
        help the weakest one by bringing points from the former to the latter and offsetting them by the lane width.
        We create two lists:
        - Left points offset to the position of the right line. These will be used to strengthen the right line if the left one
          is strong and the right one is weak.
        - Vice-versa for the right points.
 
        We execute this only if there's a big contrast between the two lines (i.e. this will not be done if we have two strong or two weak lines).
        '''
        # Create the offset lists
        new_r_x = []
        new_r_y = []
        for i in range(line_data_full_all_l.num):
            new_r_x.append(line_data_full_all_l.x[i] - x_begin_l + x_begin_r)
            new_r_y.append(line_data_full_all_l.y[i])
        new_l_x = []
        new_l_y = []
        for i in range(line_data_full_all_r.num):
            new_l_x.append(line_data_full_all_r.x[i] - x_begin_r + x_begin_l)
            new_l_y.append(line_data_full_all_r.y[i])
        # Check the lines strength
        line_weak_l = line_data_full_all_l.num < 20
        line_weak_r = line_data_full_all_r.num < 20
        line_strong_l =  line_data_full_all_l.num >= 20
        line_strong_r =  line_data_full_all_r.num >= 20
        # Apply
        if (line_weak_l and line_strong_r):
            print ("Using right points (full={} all={}) to detect left line (full={} all={})".format(line_data_full_r.num, line_data_full_all_r.num, line_data_full_l.num, line_data_full_all_l.num))
            line_data_full_all_l.append([LineData(new_l_x, new_l_y)])
        if (line_weak_r and line_strong_l):
            print ("Using left points (full={} all={}) to detect right line (full={} all={})".format(line_data_full_l.num, line_data_full_all_l.num, line_data_full_r.num, line_data_full_all_r.num))
            line_data_full_all_r.append([LineData(new_r_x, new_r_y)])
        
        # Finally, use our lists of points to create the left and right Line() objects
        self.l_line = Line(self.x_size, self.y_size, params.xm_per_pix, params.ym_per_pix, params.num_y_vals, x_begin_l, self.line_list_l, line_data_full_all_l, line_data_bottom_l, line_data_full_l)
        self.r_line = Line(self.x_size, self.y_size, params.xm_per_pix, params.ym_per_pix, params.num_y_vals, x_begin_r, self.line_list_r, line_data_full_all_r, line_data_bottom_r, line_data_full_r)
        
        # By this point, we must have two detected lines
        assert self.l_line.detected and self.r_line.detected
        
        # Fit each line separately using the provided detection points
        self.l_line.fit(params.low_pass_a)
        self.r_line.fit(params.low_pass_a)

        # Draw the lane onto the warped blank image
        fill_poly_image = self.create_empty_3_chan_for_1_chan(image_to_fit)
        self.draw_fit_area(fill_poly_image, (0, 255, 0))
        
        # Draw data information into a new image        
        fill_text_image = self.create_empty_3_chan_for_1_chan(image_to_fit)
        self.draw_width_info(fill_text_image, cv2.FONT_HERSHEY_SIMPLEX, 2.5, (255, 255, 255), 7, 3)
        self.draw_offset_info(fill_text_image, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, 2, 8)
        
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        fill_poly_image_dewarped = cv2.warpPerspective(fill_poly_image, M_inv, (self.x_size, self.y_size))
        fill_text_image_dewarped = cv2.warpPerspective(fill_text_image, M_inv, (self.x_size, self.y_size))
        
        self.result = cv2.addWeighted(self.image_undistorted, 1, fill_poly_image_dewarped, 0.3, 0)
        self.result = cv2.addWeighted(self.result, 1, fill_text_image_dewarped, 0.9, 0)
        
        self.draw_windows(self.warped_fit, windows_l, (255, 0, 0), 5)
        self.draw_windows(self.warped_fit, windows_r, (0, 255, 0), 5)
        
        self.l_line.draw_peaks(self.warped_fit, 30, (255, 0, 0), 5, -1)
        self.r_line.draw_peaks(self.warped_fit, 30, (0, 255, 0), 5, -1)
        
        self.l_line.draw_fit(self.warped_fit, (255, 165, 0), 15)
        self.r_line.draw_fit(self.warped_fit, (255, 255, 0), 15)
        
        # Draw radii information into the final image
        self.draw_radii_info(self.result, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 1)
        
print("Done.")

In [ ]:
print("Defining parameters...")

'''
This class holds the parameters that will be passed to the different
stages of the lane detection and representation pipeline.
'''
class Params():
    def __init__(self):
        # Distortion correction parameters (chessboard pattern size)
        self.calibration_nx = 9
        self.calibration_ny = 6
        
        # Input values for the perspective transformation
        self.perspective_src = np.float32([[600, 450], [680, 450], [1130, 720], [270, 720]])
        x_min = 430
        x_max = 870
        y_min = 0
        y_max = 720
        self.perspective_dst = np.float32([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])
        
        self.sobel_kernel = 3
        self.sobelx_threshold = (15, 175)
        self.angle_thresh = (0.1 * np.pi/2, 0.5 * np.pi/2)
        
        self.h_threshold = (0, 160)
        self.l_threshold = (250, 255)
        self.s_threshold = (250, 255)
        
        self.window_size_ratio = 0.1 # Ratio of the search window height to the image height
        self.stride_ratio = 1. # Ratio of the stride to the search window height
        self.search_width = 50 # Width in pixels of the search width given a previous detection guide exists
        
        self.xm_per_pix = 3.7/700 # Meters per pixel in x dimension
        self.ym_per_pix = 30/720 # Meters per pixel in y dimension
        self.num_y_vals = 101 # Num of displayed y fit values
        
        self.low_pass_a = 0.5 # Low pass filter value when smoothing previous fits
        self.max_prev_frames = 3 # Number of previous frames to be considered in a video stream

params = Params()

print("Using:\n\tcalibration_nx={}\n\tcalibration_ny={}\n\tperspective_src={}\n\tperspective_dst={}\n\twindow_size_ratio={}\n\tstride_ratio={}\n\tsearch_width={}\n\tsobel_kernel={}\n\tsobelx_threshold={}\n\tangle_thresh={}\n\th_threshold={}\n\tl_threshold={}\n\ts_threshold={}" \
      "\n\txm_per_pix={}\n\tym_per_pix={}\n\tnum_y_vals={}\n\tmax_prev_frames={}\n\tlow_pass_a={}"
      .format(params.calibration_nx, params.calibration_ny, params.perspective_src, params.perspective_dst, params.window_size_ratio, params.stride_ratio, params.search_width, params.sobel_kernel, params.sobelx_threshold, params.angle_thresh,
              params.h_threshold, params.l_threshold, params.s_threshold, params.xm_per_pix, params.ym_per_pix, params.num_y_vals, params.max_prev_frames, params.low_pass_a))

print("Done.")

In [ ]:
# Calibrate camera

mtx = None
dist = None
objpoints = []
imgpoints = []

image_paths = []
images = []
images_chess = []

images_dir = "camera_cal"
list_images = os.listdir(images_dir)
for image_name in list_images:
    image_path = os.path.join(images_dir, image_name)
    if os.path.isdir(image_path):
        continue
    
    print('Handling {0}...'.format(image_path))
    
    ret, image, image_chess, objpoints, imgpoints = findCorners(image_path, params.calibration_nx, params.calibration_ny, objpoints, imgpoints)
    
    print('  Size: {0}'.format(image.shape))
    if (not ret):
        print('  Could not find corners on {0}'.format(image_path))
    
    image_paths.append(image_path)
    images.append(image)
    images_chess.append(image_chess)

print("Calibrating camera...")

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image.shape[0:2], None, None)

print("Done.")

In [ ]:
plt.rcParams["figure.figsize"] = [9, 30]
#plt.rcParams["figure.autolayout"] = True
plt.axis('off')

num_images = len(image_paths)
num_cols = 3
num_rows = num_images

print('Displaying {0} images...'.format(num_images))
for i in range(num_images):
    k = num_cols * i + 1
    
    subplot = plt.subplot(num_rows, num_cols, k)
    subplot.imshow(images[i])
    subplot.set_title(image_paths[i])
    subplot.axis('off')
    
    if (len(images_chess[i]) != 0):
        subplot = plt.subplot(num_rows, num_cols, k + 1)
        subplot.imshow(images_chess[i])
        subplot.set_title("Chessboard")
        subplot.axis('off')
    
    image_undist = cv2.undistort(images[i], mtx, dist, None, mtx)
    
    subplot = plt.subplot(num_rows, num_cols, k + 2)
    subplot.imshow(image_undist)
    subplot.set_title("Undistorted")
    subplot.axis('off')

plt.tight_layout(w_pad=0)

print("Done.")

In [ ]:
print("Calculating perspective transform...")

M, M_inv = calculatePerspectiveTransform(params.perspective_src, params.perspective_dst)

print("Done.")

In [ ]:
images_dir = "test_images"
list_images = os.listdir(images_dir)

print("Processing images in directory '{0}':".format(images_dir))

frames = []
for image_name in list_images:
    image_path = os.path.join(images_dir, image_name)
    if os.path.isdir(image_path):
        continue
    
    image = mpimg.imread(image_path)
    print("  {0} size={1}".format(image_path, image.shape))
    
    frame = Frame(image, image_name)
    frame.process(mtx, dist, M, M_inv, params)
    
    frames.append(frame)
    
num_images = len(frames)

print("Processed {0} images.".format(num_images))

In [ ]:
plt.rcParams["figure.figsize"] = [18, 12]

num_rows = num_images
num_cols = 8

fontsize = 8
plt.axis('off')

print('Showing', num_images, 'images:')

k = 1
for frame in frames:
    subplot = plt.subplot(num_rows, num_cols, k)
    subplot.imshow(frame.image)
    subplot.set_title(frame.title, fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 1)
    subplot.imshow(frame.image_undistorted)
    subplot.set_title("Undistorted", fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 2)
    subplot.imshow(frame.warped)
    subplot.set_title("Warped", fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 3)
    subplot.imshow(frame.sobelx_binary, cmap='gray')
    subplot.set_title("Sobel X Binary", fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 4)
    subplot.imshow(frame.sobel_angle_binary, cmap='gray')
    subplot.set_title("Sobel Angle Binary", fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 5)
    subplot.imshow(frame.l_binary, cmap='gray')
    subplot.set_title("L Channel Binary", fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 6)
    subplot.imshow(frame.s_binary, cmap='gray')
    subplot.set_title("S Channel Binary", fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 7)
    subplot.imshow(frame.combined_binary, cmap='gray')
    subplot.set_title("Combined Binary", fontsize=fontsize)
    subplot.axis('off')
    
    k = k + num_cols
    
plt.tight_layout(w_pad=0, h_pad=0, rect=[0, 0, 1, 1])

print("Done.")

In [ ]:
num_cols = 3

print('Showing', num_images, 'images:')

k = 1
for frame in frames:
    x_size = frame.x_size
    y_size = frame.y_size
    
    subplot = plt.subplot(num_rows, num_cols, k)
    subplot.imshow(frame.warped_fit)
    subplot.set_title(frame.title, fontsize=fontsize)
    subplot.axis('off')
    
    subplot = plt.subplot(num_rows, num_cols, k + 1)
    subplot.plot(frame.histogram)
    subplot.set_xlim([0, x_size])
    subplot.set_title("Histogram", fontsize=fontsize)
    
    subplot = plt.subplot(num_rows, num_cols, k + 2)
    subplot.imshow(frame.result)
    subplot.set_title("Result", fontsize=fontsize)
    subplot.axis('off')
    
    k = k + num_cols

#plt.tight_layout(w_pad=0, h_pad=0, rect=[0, 0, 1, 1])

print("Done.")

In [ ]:
print("Define video pipeline...")

num_f = 0
line_list = []

def reset_globals():
    global num_f
    global line_list
    
    num_f = 0
    line_list = []
    
def process_image(image):
    global num_f
    global line_list
    
    frame = Frame(image, line_list=line_list)
    frame.process(mtx, dist, M, M_inv, params)
    
    if (len(line_list) < params.max_prev_frames):
        line_list.append((frame.l_line, frame.r_line))
    else:
        line_list = line_list[1:]
        line_list.append((frame.l_line, frame.r_line))
    
    frame.stack_thumbnails()
    
    num_f = num_f + 1
    
    return frame.result

def process_video(video_filename, video_out_filename):
    print("Processing '{0}' and saving the result into '{1}'...".format(video_filename, video_out_filename))
    
    reset_globals()
    
    video_clip = VideoFileClip(video_filename)#.subclip(0,1)

    print("  Size={0} Duration={1}s {2}FPS".format(video_clip.size, video_clip.duration, video_clip.fps))

    video_clip_out = video_clip.fl_image(process_image)
    %time video_clip_out.write_videofile(video_out_filename, audio=False)
    
    print("Done.")

print("Done.")

In [ ]:
video_out_filename = "project_video_out.mp4"
process_video("project_video.mp4", video_out_filename)

HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(video_out_filename))